<a href="https://colab.research.google.com/github/Rupkatha-dev/Employee_Burnout_Analysis/blob/main/BREAST_CANCER_DETECTION.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.
import kagglehub
awsaf49_cbis_ddsm_breast_cancer_image_dataset_path = kagglehub.dataset_download('awsaf49/cbis-ddsm-breast-cancer-image-dataset')

print('Data source import complete.')


Data source import complete.


# This Master's project for the University of Anbar,
The aim is to build a Model to detect the disease at an early phase that will assists doctors in the diagnosing phase.
Objectives:
- to detect breast cancer in The early assist Using This Model.


# We have several phases to Accomplished
# 1- Discover DataSet :
> #### https://www.kaggle.com/datasets/awsaf49/cbis-ddsm-breast-cancer-image-dataset/data
> Summary
> This CBIS-DDSM (Curated Breast Imaging Subset of DDSM) is an updated and standardized version of the Digital Database for Screening Mammography (DDSM). The DDSM is a database of 2,620 scanned film mammography studies. It contains normal, benign, and malignant cases with verified pathology information. The database's scale and ground truth validation make the DDSM a useful tool in developing and testing decision support systems. The CBIS-DDSM collection includes a subset of the DDDSM data selected and curated by a trained mammographer. The images have been decompressed and converted to DICOM format. Updated ROI segmentation and bounding boxes, and pathologic diagnosis for training data are also included. A manuscript describing how to use this dataset in detail is available at https://www.nature.com/articles/sdata2017177.

> Published research results from work in developing decision support systems in mammography are difficult to replicate due to the lack of a standard evaluation data set; most computer-aided diagnosis (CADx) and detection (CADe) algorithms for breast cancer in mammography are evaluated on private data sets or on unspecified subsets of public databases. Few well-curated public datasets have been provided for the mammography community. These include the DDSM, the Mammographic Imaging Analysis Society (MIAS) database, and the Image Retrieval in Medical Applications (IRMA) project. Although these public data sets are useful, they are limited in terms of data set size and accessibility.

> For example, most researchers using the DDSM do not leverage all its images for a variety of historical reasons. When the database was released in 1997, computational resources to process hundreds or thousands of images were not widely available. Additionally, the DDSM images are saved in non-standard compression files that require the use of decompression code that has not been updated or maintained for modern computers. Finally, the ROI annotations for the abnormalities in the DDSM were provided to indicate a general position of lesions, but not a precise segmentation for them. Therefore, many researchers must implement segmentation algorithms for accurate feature extraction. This causes an inability to compare the performance of methods directly or to replicate prior results. The CBIS-DDSM collection addresses that challenge by publicly releasing a curated and standardized version of the DDSM for evaluation of future CADx and CADe systems (sometimes referred to generally as CAD) research in mammography.

> Please note that the image data for this collection is structured such that each participant has multiple patient IDs. For example, participant 00038 has 10 separate patient IDs which provide information about the scans within the IDs (e.g. Calc-Test_P_00038_LEFT_CC, Calc-Test_P_00038_RIGHT_CC_1). This makes it appear as though there are 6,671 patients according to the DICOM metadata, but there are only 1,566 actual participants in the cohort.


# 2- Import Libraries

In [7]:
import os
import gc
import PIL
import cv2
import uuid
import shutil
import random
import glob as gb
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
%matplotlib inline

from PIL import Image
from tqdm import tqdm  # Progress bar
from scipy.special import gamma

import keras
from keras.optimizers import *
from keras.regularizers import l1_l2
from keras.utils import to_categorical
from keras.callbacks import EarlyStopping
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Input
from keras.layers import GlobalAveragePooling2D
from keras.callbacks import LearningRateScheduler
from keras.layers import Conv2D, MaxPool2D, BatchNormalization

from tensorflow.keras.metrics import *
from tensorflow.keras.callbacks import *
from tensorflow.keras.preprocessing.image import ImageDataGenerator


from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle

# Function to get the version of a module if it exists
def get_version(module):
    try:
        return module.__version__
    except AttributeError:
        return "Version info not available"

# Print versions of each module
print("os:", os.__version__ if hasattr(os, '__version__') else "OS version not available")
print("gc:", get_version(gc))
print("PIL:", get_version(PIL))
print("cv2:", cv2.__version__)
print("uuid:", get_version(uuid))
print("shutil:", get_version(shutil))
print("random:", get_version(random))
print("glob:", get_version(gb))
print("numpy:", np.__version__)
print("pandas:", pd.__version__)
print("tensorflow:", tf.__version__)
import matplotlib
print("matplotlib:", matplotlib.__version__)
import tqdm
print("tqdm:", tqdm.__version__)
import scipy
print("scipy:", scipy.__version__)
print("keras:", keras.__version__)
print("tensorflow:", tf.__version__)
import sklearn
print("sklearn:", sklearn.__version__)  # sklearn version

os: OS version not available
gc: Version info not available
PIL: 11.1.0
cv2: 4.11.0
uuid: Version info not available
shutil: Version info not available
random: Version info not available
glob: Version info not available
numpy: 1.26.4
pandas: 2.2.2
tensorflow: 2.18.0
matplotlib: 3.10.0
tqdm: 4.67.1
scipy: 1.13.1
keras: 3.8.0
tensorflow: 2.18.0
sklearn: 1.6.1


# 3- Loading DataSet

In [13]:
path = "/root/.cache/kagglehub/datasets/awsaf49/cbis-ddsm-breast-cancer-image-dataset/versions/1"

for files in os.listdir(path):
    files_dir = os.path.join(path, files)

#     print(os.path.join(path, files))
#     print(f"file {files} has {len(os.listdir(files_dir))} files")

    if files == 'jpeg':   # to pass 6774 files
        pass
    else:
        for file in os.listdir(files_dir):
            print(file)


mass_case_description_test_set.csv
dicom_info.csv
calc_case_description_train_set.csv
calc_case_description_test_set.csv
meta.csv
mass_case_description_train_set.csv


> ------

> #### We have 6 Files csv


# 4- Data Preparation

In [1]:
dicom_df = pd.read_csv("/root/.cache/kagglehub/datasets/awsaf49/cbis-ddsm-breast-cancer-image-dataset//versions/1/csv/dicom_info.csv")

NameError: name 'pd' is not defined

In [32]:
dicom_df.head(3)

,file_path,image_path,AccessionNumber,BitsAllocated,BitsStored,BodyPartExamined,Columns,ContentDate,ContentTime,ConversionType,...,SecondaryCaptureDeviceManufacturerModelName,SeriesDescription,SeriesInstanceUID,SeriesNumber,SmallestImagePixelValue,SpecificCharacterSet,StudyDate,StudyID,StudyInstanceUID,StudyTime
0,CBIS-DDSM/dicom/1.3.6.1.4.1.9590.100.1.2.12930...,CBIS-DDSM/jpeg/1.3.6.1.4.1.9590.100.1.2.129308...,NaN,16,16,BREAST,351,20160426,131732.685,WSD,...,MATLAB,cropped images,1.3.6.1.4.1.9590.100.1.2.129308726812851964007...,1,23078,ISO_IR 100,20160720.0,DDSM,1.3.6.1.4.1.9590.100.1.2.271867287611061855725...,214951.0
1,CBIS-DDSM/dicom/1.3.6.1.4.1.9590.100.1.2.24838...,CBIS-DDSM/jpeg/1.3.6.1.4.1.9590.100.1.2.248386...,NaN,16,16,BREAST,3526,20160426,143829.101,WSD,...,MATLAB,full mammogram images,1.3.6.1.4.1.9590.100.1.2.248386742010678582309...,1,0,ISO_IR 100,20160720.0,DDSM,1.3.6.1.4.1.9590.100.1.2.161516517311681906612...,193426.0
2,CBIS-DDSM/dicom/1.3.6.1.4.1.9590.100.1.2.26721...,CBIS-DDSM/jpeg/1.3.6.1.4.1.9590.100.1.2.267213...,NaN,16,16,BREAST,1546,20160503,111956.298,WSD,...,MATLAB,full mammogram images,1.3.6.1.4.1.9590.100.1.2.267213171011171858918...,1,0,ISO_IR 100,20160807.0,DDSM,1.3.6.1.4.1.9590.100.1.2.291043622711253836701...,161814.0


In [33]:
dicom_df.describe().T

,count,mean,std,min,25%,50%,75%,max
AccessionNumber,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
BitsAllocated,10237.0,1.321246e+01,3.812001,8.000000e+00,8.000000e+00,1.600000e+01,1.600000e+01,1.600000e+01
BitsStored,10237.0,1.321246e+01,3.812001,8.000000e+00,8.000000e+00,1.600000e+01,1.600000e+01,1.600000e+01
Columns,10237.0,2.179893e+03,1374.221082,6.800000e+01,4.530000e+02,2.728000e+03,3.112000e+03,5.431000e+03
ContentDate,10237.0,2.016047e+07,111.049745,2.016041e+07,2.016043e+07,2.016050e+07,2.016050e+07,2.017092e+07
ContentTime,10237.0,1.230613e+05,11391.443116,8.275551e+04,1.137396e+05,1.229359e+05,1.322114e+05,1.605079e+05
HighBit,10237.0,1.221246e+01,3.812001,7.000000e+00,7.000000e+00,1.500000e+01,1.500000e+01,1.500000e+01
InstanceNumber,10237.0,1.000000e+00,0.000000,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00
LargestImagePixelValue,10237.0,4.261229e+04,30995.808636,2.550000e+02,2.550000e+02,6.553500e+04,6.553500e+04,6.553500e+04
PatientBirthDate,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [34]:
dicom_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10237 entries, 0 to 10236
Data columns (total 38 columns):
 #   Column                                       Non-Null Count  Dtype  
---  ------                                       --------------  -----  
 0   file_path                                    10237 non-null  object 
 1   image_path                                   10237 non-null  object 
 2   AccessionNumber                              0 non-null      float64
 3   BitsAllocated                                10237 non-null  int64  
 4   BitsStored                                   10237 non-null  int64  
 5   BodyPartExamined                             10237 non-null  object 
 6   Columns                                      10237 non-null  int64  
 7   ContentDate                                  10237 non-null  int64  
 8   ContentTime                                  10237 non-null  float64
 9   ConversionType                               10237 non-null  object 
 10

In [35]:
dicom_df.SeriesDescription.unique()

array(['cropped images', 'full mammogram images', nan, 'ROI mask images'],
      dtype=object)

> ------

> #### Show Count of Images in each Type

In [36]:
dicom_df.SeriesDescription.value_counts()

,count
SeriesDescription,
cropped images,3567
ROI mask images,3247
full mammogram images,2857


> # check image path in dataset

> ------


> #### cropped images

In [37]:
cropped_images = dicom_df[dicom_df.SeriesDescription=="cropped images"].image_path
cropped_images.head()

,image_path
0,CBIS-DDSM/jpeg/1.3.6.1.4.1.9590.100.1.2.129308...
3,CBIS-DDSM/jpeg/1.3.6.1.4.1.9590.100.1.2.381187...
6,CBIS-DDSM/jpeg/1.3.6.1.4.1.9590.100.1.2.153339...
7,CBIS-DDSM/jpeg/1.3.6.1.4.1.9590.100.1.2.178994...
10,CBIS-DDSM/jpeg/1.3.6.1.4.1.9590.100.1.2.411833...


> ------

> #### full mammogram images

In [38]:
full_mammogram = dicom_df[dicom_df.SeriesDescription=="full mammogram images"].image_path
full_mammogram.head()

,image_path
1,CBIS-DDSM/jpeg/1.3.6.1.4.1.9590.100.1.2.248386...
2,CBIS-DDSM/jpeg/1.3.6.1.4.1.9590.100.1.2.267213...
11,CBIS-DDSM/jpeg/1.3.6.1.4.1.9590.100.1.2.210396...
12,CBIS-DDSM/jpeg/1.3.6.1.4.1.9590.100.1.2.749566...
15,CBIS-DDSM/jpeg/1.3.6.1.4.1.9590.100.1.2.987658...


> ------

> #### ROI images

In [39]:
roi_mask = dicom_df[dicom_df.SeriesDescription=="ROI mask images"].image_path
roi_mask.head()

,image_path
5,CBIS-DDSM/jpeg/1.3.6.1.4.1.9590.100.1.2.153339...
8,CBIS-DDSM/jpeg/1.3.6.1.4.1.9590.100.1.2.178994...
9,CBIS-DDSM/jpeg/1.3.6.1.4.1.9590.100.1.2.411833...
14,CBIS-DDSM/jpeg/1.3.6.1.4.1.9590.100.1.2.236373...
20,CBIS-DDSM/jpeg/1.3.6.1.4.1.9590.100.1.2.357008...


In [40]:
# # delete dicom_df after finished use it
del dicom_df;    gc.collect()

26

> # change directory path of images

> ------

> #### Correct Path

In [41]:
def replace_path(sample, old_path, new_path):
    return sample.replace(old_path, new_path, regex=True)

In [42]:
def plot_smaples(sample, row=15, col=15):
    plt.figure(figsize=(row, col))
    for i, file in enumerate(sample[0:5]):
        cropped_images_show = PIL.Image.open(file)
        gray_img= cropped_images_show.convert("L")
        plt.subplot(1,5,i+1)
        plt.imshow(gray_img, cmap='gray')
        plt.axis('off')
    plt.show()

In [56]:
correct_dir = "/root/.cache/kagglehub/datasets/awsaf49/cbis-ddsm-breast-cancer-image-dataset//versions/1/jpeg"

In [59]:
cropped_images = replace_path(cropped_images, "CBIS-DDSM/jpeg", correct_dir)
print('Cropped Images paths:')
print(cropped_images.iloc[0])

Cropped Images paths:
../input/cbis-ddsm-breast-cancer-image-dataset/jpeg/1.3.6.1.4.1.9590.100.1.2.129308726812851964007517874181459556304/1-172.jpg


In [60]:
plot_smaples(cropped_images, 15, 15)

FileNotFoundError: [Errno 2] No such file or directory: '../input/cbis-ddsm-breast-cancer-image-dataset/jpeg/1.3.6.1.4.1.9590.100.1.2.129308726812851964007517874181459556304/1-172.jpg'

<Figure size 1500x1500 with 0 Axes>

In [1]:
full_mammogram = replace_path(full_mammogram, "CBIS-DDSM/jpeg", correct_dir)
print('\nFull mammo Images paths:')
print(full_mammogram.iloc[0])

NameError: name 'replace_path' is not defined

In [ ]:
plot_smaples(full_mammogram)

In [ ]:
roi_mask = replace_path(roi_mask, "CBIS-DDSM/jpeg", correct_dir)
print('\nROI Mask Images paths:')
print(roi_mask.iloc[0])

In [ ]:
plot_smaples(roi_mask)

> # organize image paths

>> ---

>> By get image file name

In [ ]:
def get_image_file_name(data, new_dict):
    """
    /kaggle
    /input
    /cbis-ddsm-breast-cancer-image-dataset
    /jpeg
    /1.3.6.1.4.1.9590.100.1.2.129308726812851964007517874181459556304 [5]
    /1-172.jpg

    return path at index [5] after split depends on split('\')
    """
    for dicom in data:
        key = dicom.split('/')[4]
#         print(key)
        new_dict[key] = dicom
    print(f"the length of dataset ==> {len(new_dict.keys())}")

In [ ]:
cropped_images_dict = dict()
full_mammo_dict = dict()
roi_img_dict = dict()

get_image_file_name(cropped_images, cropped_images_dict)
get_image_file_name(full_mammogram, full_mammo_dict)
get_image_file_name(roi_mask, roi_img_dict)

> ------

> #### View keys

In [ ]:
next(iter((cropped_images.items())))

In [ ]:
next(iter((cropped_images_dict.items())))
print()
print(len(cropped_images_dict.keys()))

In [ ]:
next(iter((full_mammo_dict.items())))
print()
print(len(full_mammo_dict.keys()))

In [ ]:
next(iter((roi_img_dict.items())))
print()
print(len(roi_img_dict.keys()))

In [ ]:
del cropped_images, full_mammogram, roi_mask;    gc.collect()

> ------

> # fix image paths

In [ ]:
def fix_image_path(data):
    """Correct dicom paths to correct image paths."""
    for indx, image in enumerate(data.values):
#         print(f"Image Path: {image[11]}")

        img_name = image[11].split('/')[2]
#         print(f"Looking for key: {img_name}")  # Debugging step

        if img_name in full_mammo_dict:
            data.iloc[indx, 11] = full_mammo_dict[img_name]
        else:
            data.iloc[indx, 11] = None
#             print(f"KeyError: '{img_name}' not found in full_mammo_dict")  # Debugging step

        img_name = image[12].split('/')[2]
        if img_name in cropped_images_dict:
            data.iloc[indx, 12] = cropped_images_dict[img_name]
        else:
            data.iloc[indx, 11] = None
            # print(f"KeyError: '{img_name}' not found in cropped_images_dict")  # Debugging step

        img_name = image[13].split('/')[2]
        if img_name in roi_img_dict:
            data.iloc[indx, 13] = roi_img_dict[img_name]

        else:
            data.iloc[indx, 13] = None

> ------

> # load the mass dataset

In [ ]:
mass_train = pd.read_csv("/kaggle/input/cbis-ddsm-breast-cancer-image-dataset/csv/mass_case_description_train_set.csv")
mass_test  = pd.read_csv("/kaggle/input/cbis-ddsm-breast-cancer-image-dataset/csv/mass_case_description_test_set.csv")

In [ ]:
mass_train.head()

In [ ]:
mass_train.iloc[:, 11].head()

> ------

> #### apply to datasets

In [ ]:
fix_image_path(mass_train)

> ------

> #### check unique values in pathology column

In [ ]:
mass_train.info()

> ------

> #### rename columns

In [ ]:
mass_train = mass_train.rename(columns={'left or right breast': 'left_or_right_breast',
                                        'image view': 'image_view',
                                        'abnormality id': 'abnormality_id',
                                        'abnormality type': 'abnormality_type',
                                        'mass shape': 'mass_shape',
                                        'mass margins': 'mass_margins',
                                        'image file path': 'image_file_path',
                                        'cropped image file path': 'cropped_image_file_path',
                                        'ROI mask file path': 'ROI_mask_file_path'})
mass_train.head(5)

In [ ]:
mass_train.pathology.unique()

> ------

> #### view mass_test

In [ ]:
mass_test.head()

In [ ]:
mass_train.iloc[:, 11].head()

In [ ]:
fix_image_path(mass_test)

> ------

> #### # rename columns

In [ ]:
mass_test = mass_test.rename(columns={'left or right breast': 'left_or_right_breast',
                                      'image view': 'image_view',
                                      'abnormality id': 'abnormality_id',
                                      'abnormality type': 'abnormality_type',
                                      'mass shape': 'mass_shape',
                                      'mass margins': 'mass_margins',
                                      'image file path': 'image_file_path',
                                      'cropped image file path': 'cropped_image_file_path',
                                      'ROI mask file path': 'ROI_mask_file_path'})
# view renamed columns
mass_test.head()

> -------

> #### check datasets shape

In [ ]:
print(f'Shape of mass_train: {mass_train.shape}')
print(f'Shape of mass_test: {mass_test.shape}')

> ------

> # load the calc dataset

In [ ]:
calc_train = pd.read_csv("/kaggle/input/cbis-ddsm-breast-cancer-image-dataset/csv/calc_case_description_train_set.csv")
calc_test  = pd.read_csv("/kaggle/input/cbis-ddsm-breast-cancer-image-dataset/csv/calc_case_description_test_set.csv")

In [ ]:
calc_train.head()

In [ ]:
calc_train.iloc[:,11].head()

> --------

> #### Rename Columns

In [ ]:
calc_train = calc_train.rename(columns={'left or right breast': 'left_or_right_breast',
                                        'image view': 'image_view',
                                        'abnormality id': 'abnormality_id',
                                        'abnormality type': 'abnormality_type',
                                        'mass shape': 'mass_shape',
                                        'mass margins': 'mass_margins',
                                        'image file path': 'image_file_path',
                                        'cropped image file path': 'cropped_image_file_path',
                                        'ROI mask file path': 'ROI_mask_file_path'})
# view renamed columns
calc_train.head()

In [ ]:
fix_image_path(calc_train)

In [ ]:
calc_test.head()

In [ ]:
calc_test.iloc[:,11].head()

> --------

> #### Rename Columns

In [ ]:
calc_test = calc_test.rename(columns={'left or right breast': 'left_or_right_breast',
                                      'image view': 'image_view',
                                      'abnormality id': 'abnormality_id',
                                      'abnormality type': 'abnormality_type',
                                      'mass shape': 'mass_shape',
                                      'mass margins': 'mass_margins',
                                      'image file path': 'image_file_path',
                                      'cropped image file path': 'cropped_image_file_path',
                                      'ROI mask file path': 'ROI_mask_file_path'})
# view renamed columns
calc_test.head()

In [ ]:
fix_image_path(calc_test)

In [ ]:
print(f'Shape of mass_train: {calc_train.shape}')
print(f'Shape of mass_test: {calc_test.shape}')

> ------

> ## Show Images From All DataSets

> ------

> #### create function to display images

In [ ]:
import matplotlib.pyplot as plt
import cv2
import os
import numpy as np

def display_images(dataset, column, number):
    """Displays images in dataset, handling missing files and converting formats."""

    # create figure and axes
    fig, axes = plt.subplots(1, number, figsize=(15, 5))

    # Loop through rows and display images
    for index, (i, row) in enumerate(dataset.head(number).iterrows()):
        image_path = row[column]

       # Check if image_path is valid (not None) and exists
        if image_path is None or not os.path.exists(image_path):
            # print(f"File not found or invalid path: {image_path}")
            continue

        image = cv2.imread(image_path)

        # Handle case when image can't be read
        if image is None:
            # print(f"Error reading image: {image_path}")
            continue

        # Convert BGR to RGB if needed (for correct color display)
        if len(image.shape) == 3 and image.shape[2] == 3:
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

        ax = axes[index]
        ax.imshow(image, cmap='gray' if len(image.shape) == 2 else None)
        ax.set_title(f"{row['pathology']}")
        ax.axis('off')
        print(np.array(image).shape)

    plt.tight_layout()
    plt.show()

> ------

> ### Mass_Train

In [ ]:
print('Full Mammograms:\n')
display_images(mass_train, 'image_file_path', 5)
print('Cropped Mammograms:\n')
display_images(mass_train, 'cropped_image_file_path', 5)
print('ROI_mask:\n')
display_images(mass_train, 'ROI_mask_file_path', 5)

> ------

> ### Mass_Test

In [ ]:
print('Full Mammograms:\n')
display_images(mass_test, 'image_file_path', 5)
print('Cropped Mammograms:\n')
display_images(mass_test, 'cropped_image_file_path', 5)
print('ROI_mask:\n')
display_images(mass_test, 'ROI_mask_file_path', 5)

> ------

> ### Calc_Train

In [ ]:
print('Full Mammograms:\n')
display_images(calc_train, 'image_file_path', 5)
print('Cropped Mammograms:\n')
display_images(calc_train, 'cropped_image_file_path', 5)
print('ROI_mask:\n')
display_images(calc_train, 'ROI_mask_file_path', 5)

> ------

> ### Calc_Test

In [ ]:
print('Full Mammograms:\n')
display_images(calc_test, 'image_file_path', 5)
print('Cropped Mammograms:\n')
display_images(calc_test, 'cropped_image_file_path', 5)
print('ROI_mask:\n')
display_images(calc_test, 'ROI_mask_file_path', 5)

# 5- Data Preprocessing

> ------

> ## Merge datasets

In [ ]:
# full_dataset = pd.concat([mass_train, mass_test, calc_train, calc_test], axis=0)
# full_dataset = pd.concat([mass_train, mass_test], axis=0)
full_dataset = pd.concat([calc_train, calc_test], axis=0)

In [ ]:
del mass_train, mass_test, calc_train, calc_test;    gc.collect()

> ------

> ## Show Images size

In [ ]:
# from tqdm import tqdm  # Progress bar

# def show_shape(dataset, path):
#     """Show Images Shapes"""
#     shapes = []

#     paths = np.array(dataset[path])

#     # Use tqdm to track progress
#     for file_path in tqdm(paths, total=len(paths)):
#         # Check if the file != None
#         if file_path:
#             # Read the images
#             image = cv2.imread(file_path)
#             shapes.append(image.shape)
#         else:
#             pass

#     return shapes

> -----

> ### For Full_Mammograms

In [ ]:
# full_mammograms_images = show_shape(full_dataset, 'image_file_path')
# cropped_images         = show_shape(full_dataset, 'cropped_image_file_path')

In [ ]:
# print("Full_mammogram_images:")
# print(min(full_mammograms_images))
# print(max(full_mammograms_images))
# print()
# print("Cropped_images:")
# print(min(cropped_images))
# print(max(cropped_images))

* > ## Create a binary mapper

In [ ]:
class_mapper = {'MALIGNANT': 1, 'BENIGN': 0, 'BENIGN_WITHOUT_CALLBACK': 0}

> ## Define the target size

In [ ]:
target_size = (224, 224, 3)

In [ ]:
# Apply class mapper to pathology column
full_dataset['labels'] = full_dataset['pathology'].replace(class_mapper).infer_objects(copy=False)

full_images = np.array(full_dataset[full_dataset["image_file_path"].notna()]["image_file_path"].tolist())
full_labels = np.array(full_dataset[full_dataset["image_file_path"].notna()]["labels"].tolist())

In [ ]:
len(full_images)

In [ ]:
# If full_labels is a NumPy array, convert it to a Pandas series
full_labels_series = pd.Series(full_labels)

# Count the occurrences of each class
label_counts = full_labels_series.value_counts()

# Assuming 0 = benign and 1 = malignant
benign_count = label_counts.get(0, 0)
malignant_count = label_counts.get(1, 0)

print(f"Benign images: {benign_count}")
print(f"Malignant images: {malignant_count}")

* > ## Check the number of classes

In [ ]:
num_classes = len(full_dataset['labels'].unique())
num_classes

In [ ]:
class_names = ['Benign', 'Malignant']

In [ ]:
# # Check the distribution of labels
# label_counts = full_dataset['labels'].value_counts()
# print(label_counts)

In [ ]:
# Remove a single file
# file_path = '/kaggle/working/CBIS-DDSM-BCID-and-BCD.zip'
# if os.path.exists(file_path):
#     os.remove(file_path)
#     print(f"Removed file: {file_path}")
# else:
#     pass
#     print(f"File does not exist: {file_path}")

# Remove a directory and its contents
# dir_path = '/kaggle/working/'
# if os.path.exists(dir_path):
#     shutil.rmtree(dir_path)
#     print(f"Removed directory: {dir_path}")
# else:
#     pass
#     print(f"Directory does not exist: {dir_path}")

# Path to the working directory
dir_path = '/kaggle/working/'

# # Loop through all files and subdirectories in the working directory
for filename in os.listdir(dir_path):
    file_path = os.path.join(dir_path, filename)
    try:
        if os.path.isfile(file_path) or os.path.islink(file_path):
            os.unlink(file_path)  # Remove the file or symbolic link
        elif os.path.isdir(file_path):
            shutil.rmtree(file_path)  # Remove the directory and its contents
    except Exception as e:
        print(f'Failed to delete {file_path}. Reason: {e}')

print("All files and subdirectories in '/kaggle/working/' have been removed.")


In [ ]:
def images_count():
    zero_class_count = len(os.listdir("/kaggle/working/merged_images/0"))
    one_class_count  = len(os.listdir("/kaggle/working/merged_images/1"))

    print(f"Number of images in class 0: {zero_class_count}")
    print(f"Number of images in class 1: {one_class_count}")

In [ ]:
# Define a function for data augmentation
def augment_image(image):
    # Apply data augmentation using tf.image functions
    image = tf.image.random_flip_left_right(image)
#     image = tf.image.random_flip_up_down(image)
    image = tf.image.random_brightness(image, max_delta=0.3)
    image = tf.image.random_contrast(image, lower=0.8, upper=1.2)
    image = tf.image.random_saturation(image, lower=0.8, upper=1.2)
    return image

# Function to resize image to (224, 224, 3)
def resize_image(image_tensor):
    return tf.image.resize(image_tensor, [224, 224])

# Function to balance classes by augmenting images
def copy_images_with_unique_filenames(images, labels, source, destination, target_count=None):
    """
    Copy images from source to destination in subfolders '0' and '1',
    ensuring unique filenames and applying data augmentation and balancing.
    """
    benign_images = 0
    malignant_images = 0
    skipped_images = []

    # Create the destination subfolders '0' and '1'
    category_dest_dir_zero = os.path.join(destination, '0')
    os.makedirs(category_dest_dir_zero, exist_ok=True)

    category_dest_dir_one = os.path.join(destination, '1')
    os.makedirs(category_dest_dir_one, exist_ok=True)

    benign_images_list = []
    malignant_images_list = []

    for i, (image, label) in enumerate(zip(images, labels)):
#         img_name = data_frame.REFNUM[i]
#         abs_path = os.path.join(source, img_name + '.pgm')

        if os.path.exists(image):
            try:
                # Generate a unique filename
                filename = os.path.basename(image)
                unique_filename = f"{uuid.uuid4().hex}_{filename}"

                # Open the image using PIL
                with Image.open(image) as img:
                    # Convert the image to RGB mode (for saving as JPEG)
                    img = img.convert('RGB')
                    # Augment the image (convert it to a Tensor first)
                    img_tensor = tf.convert_to_tensor(img)
                    # Resize the image to (224, 224, 3)
                    resized_img_tensor = resize_image(img_tensor)
                    augmented_image_tensor = augment_image(resized_img_tensor)
                    # Convert Tensor back to PIL image for saving
                    augmented_image = tf.keras.preprocessing.image.array_to_img(augmented_image_tensor)

                    if label == 0:
                        benign_images_list.append(unique_filename)
                        dest_path = os.path.join(category_dest_dir_zero, unique_filename)
#                         augmented_image.save(dest_path, 'JPEG')
                        augmented_image.save(dest_path, 'JPEG')
                        benign_images += 1

                    elif label == 1:
                        malignant_images_list.append(unique_filename)
                        dest_path = os.path.join(category_dest_dir_one, unique_filename)
#                         augmented_image.save(dest_path, 'JPEG')
                        augmented_image.save(dest_path, 'JPEG')
                        malignant_images += 1

#                 del img, img_tensor, resized_img_tensor, augmented_image_tensor, augmented_image
#                 gc.collect()
            except Exception as e:
                print(f"Error copying image {image}: {e}")
                skipped_images.append(image)
        else:
            print(f"Image not found: {image}")
            skipped_images.append(image)

    # If balancing is needed, duplicate/augment images from the smaller class
    benign_count = len(benign_images_list)
    malignant_count = len(malignant_images_list)

    if benign_count < malignant_count:
#         augment_and_save_images(benign_images_list, category_dest_dir_zero, target_count - benign_count)
        augment_and_save_images(benign_images_list, category_dest_dir_zero, malignant_count - benign_count)

    elif malignant_count < benign_count:
        augment_and_save_images(malignant_images_list, category_dest_dir_one, benign_count - malignant_count)
    print("data balancing")
    images_count()
    augment_and_save_images(benign_images_list, category_dest_dir_zero, target_count)
    augment_and_save_images(malignant_images_list, category_dest_dir_one, target_count)
    print("data augmentation")
    images_count()

    print(f"\nCopying complete.")
    print(f"Benign images copied (label 0): {benign_images}")
    print(f"Benign count (label 0): {benign_count}")
    print(f"Malignant images copied (label 1): {malignant_images}")
    print(f"Malignant count (label 1): {malignant_count}")
    print(f"Total skipped images: {len(skipped_images)}")
    if skipped_images:
        print("Skipped images:")
        for img in skipped_images:
            print(img)

    del skipped_images, benign_images_list, malignant_images_list
    gc.collect()

# Function to augment and save images to balance the dataset
def augment_and_save_images(images_list, destination_dir, num_augments):
    """
    Augment and save images to balance the dataset.
    """
    for i in range(num_augments):
        img_name = random.choice(images_list)
        abs_path = os.path.join(destination_dir, img_name)

        try:
            with Image.open(abs_path) as img:
                img = img.convert('RGB')
                # Augment the image
                img_tensor = tf.convert_to_tensor(img)
                # Resize the image
#                 resized_img_tensor = resize_image(img_tensor)
                augmented_image_tensor = augment_image(img_tensor)
                # Convert Tensor back to PIL image for saving
                augmented_image = tf.keras.preprocessing.image.array_to_img(augmented_image_tensor)
                # Remove the original extension from img_name 1-285.jpg --> 1-285
                img_name_without_ext = os.path.splitext(img_name)[0]
                # Save augmented image with a unique name
                augmented_image.save(os.path.join(destination_dir, img_name_without_ext + f'_aug{i}.jpg'), 'JPEG')

#                 del img, img_tensor, augmented_image_tensor, augmented_image, img_name_without_ext
#                 gc.collect()
        except Exception as e:
            print(f"Error augmenting image {abs_path}: {e}")

# Example usage
source_dir = "/kaggle/input/mias-mammography/all-mias"
destination_dir = "/kaggle/working/merged_images"

# target_count=0 meaning no Augmentation, There's just Data-Balance
target_count = int((len(full_labels) * 5) // 2)
copy_images_with_unique_filenames(full_images, full_labels, source_dir, destination_dir, target_count)

In [ ]:
target_count

In [ ]:
images_count()

In [ ]:
# del full_images, full_labels, full_dataset;    gc.collect()

 * > --------

In [ ]:
# import tensorflow as tf

# # Step 1: Define the Data Augmentation Layer
# # data_augmentation = tf.keras.Sequential([
# #     tf.keras.layers.RandomFlip('horizontal'),  # Randomly flip images horizontally and vertically
# #     tf.keras.layers.RandomRotation(0.2),  # Randomly rotate images by up to 20%
# #     tf.keras.layers.RandomZoom(0.2),  # Randomly zoom into images by up to 20%
# #     tf.keras.layers.RandomContrast(0.2),  # Randomly adjust contrast by up to 20%
# #     tf.keras.layers.RandomTranslation(0.1, 0.0),  # Randomly translate images by up to 10% horizontally and vertically
# # ])

# # Step 2: Load and Prepare Your Datasets
# data_dir = '/kaggle/working/merged_images'  # Update with your dataset path

# train_generator = tf.keras.preprocessing.image_dataset_from_directory(
#     data_dir,
#     labels='inferred',
#     label_mode='categorical',
# #     batch_size=13,
#     image_size=(224, 224),
#     validation_split=0.3,
#     subset='training',
#     seed=50,
#     shuffle=True
# )

# validation_generator = tf.keras.preprocessing.image_dataset_from_directory(
#     data_dir,
#     labels='inferred',
#     label_mode='categorical',
# #     batch_size=13,
#     image_size=(224, 224),
#     validation_split=0.3,
#     subset='validation',
#     seed=5,
#     shuffle=True
# )

# # Step 3: Apply Augmentation to the Training Dataset
# # train_generator = train_generator.map(
# #     lambda x, y: (data_augmentation(x, training=True), y),
# #     num_parallel_calls=tf.data.experimental.AUTOTUNE
# # )

# # Step 4: Prefetch to Improve Performance
# train_generator = train_generator.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)
# validation_generator = validation_generator.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)

In [ ]:
import tensorflow as tf

# # Step 1: Define the Data Augmentation Layer
# data_augmentation = tf.keras.Sequential([
#     tf.keras.layers.RandomFlip('horizontal'),  # Randomly flip images horizontally
#     tf.keras.layers.RandomRotation(0.2),       # Randomly rotate images by up to 20%
#     tf.keras.layers.RandomZoom(0.2),           # Randomly zoom into images by up to 20%
#     tf.keras.layers.RandomContrast(0.2),        # Randomly adjust contrast by up to 20%
#     tf.keras.layers.RandomTranslation(0.1, 0.0),# Randomly translate images by up to 10% horizontally
# ])

# Step 2: Load and Prepare Your Datasets
data_dir = '/kaggle/working/merged_images'  # Update with your dataset path

# Create a dataset for the entire data to use for split
full_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    data_dir,
    labels='inferred',
    label_mode='categorical',
    # image_size=(224, 224),
    image_size=(224, 224),
    seed=30,
    shuffle=True,
    batch_size=13  # Set your desired batch size
)
# Calculate the total number of samples
total_samples = tf.data.experimental.cardinality(full_dataset).numpy()

# Step 3: Split the dataset into train, validation, and test sets
train_size = int(0.7 * total_samples)                 # 70% for training
# val_size   = int(0.15 * total_samples)                # 20% for validation
# test_size = total_samples - train_size - val_size     # 10% for testing
test_size = total_samples - train_size                # 30% for testing

# Create train, validation, and test datasets
train_dataset       = full_dataset.take(train_size)
# validation_dataset  = full_dataset.skip(train_size).take(val_size)
# test_dataset        = full_dataset.skip(train_size + val_size)
test_dataset        = full_dataset.skip(train_size)

# Step 4: Apply Augmentation to the Training Dataset
# train_dataset = train_dataset.map(
#     lambda x, y: (data_augmentation(x, training=True), y),
#     num_parallel_calls=tf.data.experimental.AUTOTUNE
# )

# Step 5: Prefetch to Improve Performance
train_dataset      = train_dataset.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)
# validation_dataset = validation_dataset.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)
test_dataset       = test_dataset.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)

# Print the number of samples in each dataset
print(f"Train samples:      {train_size}     batches(8) ==> {train_size*13}")
# print(f"Validation samples: {val_size}       batches(13) ==> {val_size*13}")
print(f"Test samples:       {test_size}      batches(8) ==> {test_size*13}")

# del full_dataset, total_samples, train_size, val_size, test_size;    gc.collect()
del full_dataset, total_samples, train_size, test_size;    gc.collect()

# Import Transfer Learning Architectures

> ## Extract features with InceptionV3

In [ ]:
from tensorflow.keras.applications import (ResNet50,
                                           Xception,
                                           VGG19,
                                           InceptionV3,
                                           DenseNet121,
                                           NASNetMobile,
                                           MobileNetV2,
                                           MobileNet,
                                           EfficientNetV2B0,
                                           EfficientNetV2S,
                                           EfficientNetV2L,
                                           ConvNeXtBase)

def create_model(model_name, trainable_layers, dropout_value, save_weights=False):
    model_dict = {
        "ResNet50":         ResNet50,
        "Xception":         Xception,
        "VGG19":         VGG19,
        "InceptionV3":      InceptionV3,
        "DenseNet121":      DenseNet121,
        "NASNetMobile":     NASNetMobile,
        "MobileNetV2":      MobileNetV2,
        "MobileNet":        MobileNet,
        "EfficientNetV2B0": EfficientNetV2B0,
        "EfficientNetV2S":  EfficientNetV2S,
        "EfficientNetV2L":  EfficientNetV2L,
        "ConvNeXtBase":     ConvNeXtBase,
    }

    if model_name not in model_dict:
        raise ValueError(f"Model {model_name} is not supported.")

    # Load the base model without the top layers
    base_model = model_dict[model_name](weights='imagenet' if not save_weights else None,
                                        include_top=False,
                                        input_shape=(224, 224, 3))

    # Freeze all layers initially
    for layer in base_model.layers:
        layer.trainable = False

    # Calculate the index to start unfreezing layers
    from_index = int(np.round((len(base_model.layers) - 1) * (1.0 - trainable_layers / 100.0)))

    # Unfreeze layers from the calculated index onwards
    for layer in base_model.layers[from_index:]:
        layer.trainable = True

    # Add custom layers on top (Upper-Layers)
    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(1024, activation='relu')(x)
    x = Dropout(dropout_value)(x)
    predictions = Dense(2, activation='softmax')(x)  # Assuming binary classification

    model = Model(inputs=base_model.input, outputs=predictions)

    # Clear the base model from memory if needed (optional)
    del model_dict, base_model, from_index, x, predictions;    gc.collect()
    return model

In [ ]:
from multiprocessing import cpu_count
n_cores = cpu_count()
print(f'Number of Logical CPU cores: {n_cores}')

In [ ]:
# Create directories if they do not exist
def make_file():
    os.makedirs(CHECKPOINTS_DIR, exist_ok=True)
    os.makedirs(LOGS_DIR, exist_ok=True)

In [ ]:
from keras import backend as K

def reset_memory():
    K.clear_session()
    gc.collect()

In [ ]:
PROJECT_PATH    = "/kaggle/working/"
CHECKPOINTS_DIR = os.path.join(PROJECT_PATH, "Checkpoints")
LOGS_DIR        = os.path.join(PROJECT_PATH, "Logs")

# Define the width you want for centering
WIDTH = 162  # For example, assuming 80 characters wide terminal or display

In [ ]:
def run_model(trainable_layers,
              optimizer_class,
              batch_size_value,
              dropout_value,
              model_name=None,
              save=False,
              epochs=5,
              patience=2):

    if model_name is None:
        models = ["ConvNeXtBase"]
        # models = ["ResNet50"]
    else:
        models = [model_name]  # Train only the specific model passed in

    best_score = float('-inf')  # Initialize best score to negative infinity
    best_model_info = None      # To store the best model information

    for model_name in models:
        # Clear any existing Keras session to free up memory
        tf.keras.backend.clear_session();    gc.collect()  # Optional: use garbage collection to ensure no memory leaks

        model = create_model(model_name, trainable_layers, dropout_value)  # Create the model
        optimizer_value = optimizer_class()  # Instantiate the optimizer

        model.compile(optimizer=optimizer_value,
                      loss='categorical_crossentropy',
                      metrics=['accuracy', Precision(name='precision'), Recall(name='recall')])  # Compile the model

        trainable_params = sum([tf.keras.backend.count_params(w) for w in model.trainable_weights])
        print('')
        output = (f"Training model: {model_name}   "
                  f"Layers: {len(model.layers)}   "
                  f"Parameters: {model.count_params()}   "
                  f"Trainable parameters: {trainable_params}")
        print(output.center(WIDTH))
        print(f'#%% {"-" * 97} %'.center(WIDTH))

        # Prepare file paths if saving is enabled
        if save:
            solution = [trainable_layers, optimizer_value.__class__.__name__, batch_size_value, dropout_value]
            keyword = f"{model_name}-" + "-".join([str(el) for el in solution])
            checkpointPath = os.path.join(PROJECT_PATH, "Checkpoints", keyword) + ".weights.h5"
            csvLogPath = os.path.join(PROJECT_PATH, "Logs", keyword) + ".csv"
            make_file()  # Create directories

        # Set common callbacks
        callbacks = [
            TerminateOnNaN(),
            EarlyStopping(monitor="accuracy", mode="max", patience=patience)
        ]

        # Add saving-related callbacks if saving is enabled
        if save:
            callbacks += [
                ModelCheckpoint(checkpointPath,
                                save_best_only=True,
                                save_weights_only=True,
                                monitor="accuracy",
                                mode="max",
                                verbose=0),
                CSVLogger(csvLogPath, append=True)
            ]

        # Train the model
        history = model.fit(
            train_dataset,
            # validation_data=validation_dataset,
            batch_size=batch_size_value,
            epochs=epochs,
            callbacks=callbacks
        )

        if save:
            return model, history

        # Calculate the score based on metrics
        # val_acc = history.history['val_accuracy'][-1]     # Last validation accuracy
        # val_precision = history.history['precision'][-1]  # Last precision
        # val_recall = history.history['recall'][-1]        # Last recall
        score = history.history['accuracy'][-1]
        # score = (val_acc + val_precision + val_recall) / 3  # Average score

        # Check if the current score is better than the best score
        if score > best_score:
            best_score = score  # Update best score
            best_model_info = {
                'model_name': model_name,
                'trainable_layers': trainable_layers,
                'optimizer': optimizer_value.__class__.__name__,
                'batch_size': batch_size_value,
                'dropout_value': dropout_value
            }

        # Clear the session after model is trained
        reset_memory()
        print(f"Model {model_name} cleared from memory".center(WIDTH))
        print()

        # After each iteration, clear the session and use garbage collection
        # Delete model to release memory
        # del model, optimizer_value, history, score, trainable_params;    gc.collect()

    # return best_score, best_model_info  # Return best score and details of best model
        return model  # Return best score and details of best model

# Apply Arctic Puffin Optimization

In [ ]:
# from tensorflow.keras.optimizers import SGD, Adam, Nadam, RMSprop, Adadelta, Adagrad

# # Define your ranges for the hyperparameters
# ranges = {
#     "Trainable_Layers": np.arange(2, 11, 2),
# #     "Optimizers":       np.array([SGD, Adam, Nadam, RMSprop, Adadelta, Adagrad]),
#     "Optimizers":       np.array([SGD, Adam, Nadam, RMSprop]),
#     "Batch_Sizes":      np.array([8, 13, 16]),
#     "Dropout":          np.arange(0, 0.3, 0.1),
#     # "Model_Name":       np.array(["EfficientNetV2B0", "ResNet50", "ConvNeXtBase"])
#     # "Model_Name":       np.array(["EfficientNetV2B0", "ResNet50", "MobileNet", "DenseNet121"])
#     "Model_Name":       np.array(["ResNet50", "ConvNeXtBase"])
# }

# # """
# # How Many Possible Solutions Are There?

# # Let’s calculate the total number of solutions by multiplying the number of options for each hyperparameter:

# # - Trainable Layers: 10 options (0, 1, ..., 9).
# # - Optimizer:         6 options (SGD, Adam, Nadam, RMSprop, Adadelta, Adagrad).
# # - Batch Size:        3 options (8, 13, 16).
# # - Dropout:           3 options (0, 0.1, 0.2).
# # - Model Name:        4 options (ResNet50, MobileNet, DenseNet121, EfficientNetV2B0).

# # The total number of possible solutions is:

# # 10 × 6 × 3 × 3 × 4 =  2,160 possible solutions.
# # """

In [ ]:
# def initialize_population(Search_agents_no, dim, ub, lb):
#     """
#     Initialize the first population of search agents.

#     Parameters:
#     Search_agents_no (int): The number of puffins (solutions) in the population.
#     dim (int): The number of dimensions (hyperparameters).
#     ub (list): Upper bounds for each dimension.
#     lb (list): Lower bounds for each dimension.

#     Returns:
#     Positions (np.array): Initialized population matrix with shape (SearchAgents_no, dim).
#     """

#     # Boundary_no is the length of upper bounds, which tells if it's a single boundary or multiple for each dimension.
#     Boundary_no = len(ub)

#     # Initialize positions
#     Positions = np.zeros((Search_agents_no, dim))

#     # If all variables have the same bounds
#     if Boundary_no == 1:
#         Positions = np.random.rand(Search_agents_no, dim) * (ub - lb) + lb
#     else:
#         # If each variable has a different bound
#         for i in range(dim):
#             ub_i = ub[i]
#             lb_i = lb[i]

#             Positions[:, i] = np.random.rand(Search_agents_no) * (ub_i - lb_i) + lb_i

#     return Positions

In [ ]:
# def index_values(solution):

#     # Extract hyperparameters from solution
#     trainable_layers_index = int(np.round(solution[0]))  # Round for trainable layers index
#     optimizer_index        = int(np.round(solution[1]))  # Round for optimizer index
#     batch_size_index       = int(np.round(solution[2]))  # Round for batch size index
#     dropout_index          = int(np.round(solution[3]))  # Round for batch size index
#     model_index            = int(np.round(solution[4]))  # Round for batch size index

#     # Ensure indices are within valid ranges
#     trainable_layers_index = np.clip(trainable_layers_index, 0, len(ranges["Trainable_Layers"]) - 1)
#     optimizer_index        = np.clip(optimizer_index,        0, len(ranges["Optimizers"]) - 1)
#     batch_size_index       = np.clip(batch_size_index,       0, len(ranges["Batch_Sizes"]) - 1)
#     dropout_index          = np.clip(dropout_index,          0, len(ranges["Dropout"]) - 1)
#     model_index            = np.clip(model_index,            0, len(ranges["Model_Name"]) - 1)

#     # Get the values based on indices
#     trainable_layers_value = ranges["Trainable_Layers"][trainable_layers_index]
#     optimizer_value        = ranges["Optimizers"][optimizer_index]
#     batch_size_value       = ranges["Batch_Sizes"][batch_size_index]
#     dropout_value          = ranges["Dropout"][dropout_index]
#     model_value            = ranges["Model_Name"][model_index]

#     return trainable_layers_value, optimizer_value, batch_size_value, dropout_value, model_value

In [ ]:
# # Example objective function (replace with your actual objective function)
# def fitness_func(solution):

#     trainable_layers_value,\
#     optimizer_value,\
#     batch_size_value,\
#     dropout_value,\
#     model_value = index_values(solution)

#     score, configs = run_model(trainable_layers_value,
#                                optimizer_value,
#                                batch_size_value,
#                                dropout_value,
#                                model_value)

#     # Center the separator
#     print(f'#%%{"-" * 36}%'.center(WIDTH))

#     # Center the validation accuracy
#     val_accuracy_output = f" val_accuracy: {score}".center(WIDTH)

#     # Center each configuration item and join them with newline characters
#     config_output = "\n".join([str(f'{key}: {value}').center(WIDTH) for key, value in configs.items()])

#     # Print the combined output centered
#     print(val_accuracy_output + "\n" + config_output)

#     # Center the separator again
#     print(f'#%%{"-" * 36}%'.center(WIDTH))

#     return score

In [ ]:
# # Step 1: Create a 4GB swap file
# !fallocate -l 4G /kaggle/working/swapfile

# # Step 2: Set the correct permissions
# !chmod 600 /kaggle/working/swapfile

# # Step 3: Set up the swap space
# !mkswap /kaggle/working/swapfile

# # Step 4: Enable the swap
# !swapon /kaggle/working/swapfile

# # Step 5: Verify swap is enabled
# !swapon --show

# # Step 6: Check memory usage (RAM + Swap)
# !free -h

In [ ]:
# # Levy flight function
# def levy(dim, beta=1.5):
#     sigma = (gamma(1 + beta) * np.sin(np.pi * beta / 2) /
#              (gamma((1 + beta) / 2) * beta * 2 ** ((beta - 1) / 2))) ** (1 / beta)
#     u = np.random.randn(dim) * sigma
#     v = np.random.randn(dim)
#     step = u / np.abs(v) ** (1 / beta)
#     return step

# # SpaceBound ensures that the positions stay within the bounds
# def space_bound(pos, ub, lb):
#     return np.clip(pos, lb, ub)

# # Arctic Puffin Optimization algorithm (for maximization)
# def APO(N, T, lower_bound, upper_bound, dim, fitness_func):

#     # This function initialize the first population of search agents
#     Population_Pos = initialize_population(N, dim, upper_bound, lower_bound)
#     Population_Fit = np.zeros(N)

#     # This function Evaluate(Fitness) the populations of search agents ==> (Objective Function)
#     for i in range(N):
#         Population_Fit[i] = fitness_func(Population_Pos[i, :])
#         print(f"Solution: {i+1} is Completed".center(WIDTH))
#         print()

#     # Initialize the best fitness and best position (Maximization)
#     BestF = -float('inf')  # Initialize as negative infinity for maximization
#     BestX = None

#     # Find the initial best position and fitness
#     for i in range(N):
#         if Population_Fit[i] > BestF:  # Use '>' for maximization
#             BestF = Population_Fit[i]
#             BestX = Population_Pos[i, :].copy()

#     curve = np.zeros(T)
#     #%% -----------------------------%  Main iteration loop   %----------------------------------%
#     for It in range(T):
#         for i in range(N):
#             theta1 = (1 - It / T)
#             B = 2 * np.log(1 / np.random.rand()) * theta1

#             #%% -----------------------------%  1.Aerial Flight Stage   %-----------------------------------%
#             if B > 0.5:
#                 while True:
#                     K = [j for j in range(N) if j != i]
#                     RandInd = random.choice(K)
#                     step1 = Population_Pos[i, :] - Population_Pos[RandInd, :]
#                     if np.linalg.norm(step1) != 0 and not np.array_equal(Population_Pos[i, :], Population_Pos[RandInd, :]):
#                         break

#                 #%% -----------------------------%  1.1 Aerial search.   %----------------------------------%
#                 # Aerial Search (Levy Flight)
#                 ALPHA = np.random.randn(dim)
#                 R = np.round(0.5 * (0.05 + np.random.rand())) * ALPHA
#                 Y = Population_Pos[i, :] + step1 * levy(dim)  + R

#                 #%% -----------------------------%  1.2 Swooping predation. %-------------------------------%
#                 # Swooping Predation
#                 R = np.random.rand(dim)
#                 step2 = (R - 0.5) * np.pi
#                 S = np.tan(step2)
#                 Z = Y * S

#                 # Ensure solutions remain within bounds
#                 Y = space_bound(Y, upper_bound, lower_bound)
#                 Z = space_bound(Z, upper_bound, lower_bound)

#                 # Fitness evaluation of new positions
#                 NewPop = np.vstack([Y, Z])

#                 Y_FITNESS = fitness_func(Y)
#                 print(f"Y_FITNESS for {i+1} solution {It+1} iteration".center(WIDTH))

#                 Z_FITNESS = fitness_func(Z)
#                 print(f"Z_FITNESS for {i+1} solution {It+1} iteration".center(WIDTH))

#                 NewPopfit = np.array([Y_FITNESS, Z_FITNESS])
#                 print(f"Flight Stage Complete for {i+1} solution {It+1} iteration".center(WIDTH))
#                 print('')

#                 # Find the best new population
#                 # 1-scores = [97, 95], ==> argsort(ascending) = [1, 0]
#                 # 2-scores = [95, 97], ==> argsort(ascending) = [0, 1]
#                 sorted_indexes = np.argsort(NewPopfit)
#                 # 1-NewPop[0, :] ==> Y
#                 # 2-NewPop[1, :] ==> Z
#                 newPopPos = NewPop[sorted_indexes[-1], :]

#             #%% -----------------------------%  2.Underwater Foraging Stage   %----------------------------------%
#             else:
#                 F = 0.5
#                 K = [j for j in range(N) if j != i]
#                 RandInd = random.sample(K, 3)

#                 while True:
#                     step1 = Population_Pos[RandInd[1], :] - Population_Pos[RandInd[2], :]
#                     if np.linalg.norm(step1) != 0 and RandInd[1] != RandInd[2]:
#                         break

#                 #%% -----------------------------%  2.1	Gathering foraging   %----------------------------------%
#                 if np.random.rand() < 0.5:
#                     W = Population_Pos[RandInd[0], :] + F * step1
#                 else:
#                     W = Population_Pos[RandInd[0], :] + F * step1 * levy(dim)

#                 #%% -----------------------------%  2.2Intensifying search   %----------------------------------%
#                 f = (0.1 * (np.random.rand() - 1) * (T - It)) / T
#                 Y = (1 + f) * W

#                 #%% --------------------------%  2.3Underwater Foraging Stage  %--------------------------------%
#                 while True:
#                     rand_leader_index1 = random.randint(0, N - 1)
#                     rand_leader_index2 = random.randint(0, N - 1)
#                     X_rand1 = Population_Pos[rand_leader_index1, :]
#                     X_rand2 = Population_Pos[rand_leader_index2, :]
#                     step2 = X_rand1 - X_rand2

#                     if np.linalg.norm(step2) != 0 and not np.array_equal(X_rand1, X_rand2):
#                         break

#                 Epsilon = np.random.uniform(0, 1)
#                 if np.random.rand() < 0.5:
#                     Z = Population_Pos[i, :] + Epsilon * step2
#                 else:
#                     Z = Population_Pos[i, :] + F * levy(dim) * step2

#                 NewPop = np.vstack([W, Y, Z])

#                 W_FITNESS = fitness_func(W)
#                 print(f"W_FITNESS for {i+1} solution {It+1} iteration".center(WIDTH))

#                 Y_FITNESS = fitness_func(Y)
#                 print(f"Y_FITNESS for {i+1} solution {It+1} iteration".center(WIDTH))

#                 Z_FITNESS = fitness_func(Z)
#                 print(f"Z_FITNESS for {i+1} solution {It+1} iteration".center(WIDTH))

#                 NewPopfit = np.array([W_FITNESS, Y_FITNESS, Z_FITNESS])
#                 print(f"Underwater Stage Complete for: {i+1} solution, {It+1} iteration".center(WIDTH))
#                 print('\n\n')
#                 # Find the best new population
#                 # 1-scores = [90, 80, 97], ==> argsort(ascending) = [1, 0, 2]
#                 # 2-scores = [97, 80, 90], ==> argsort(ascending) = [1, 2, 0]
#                 # 3-scores = [90, 97, 80], ==> argsort(ascending) = [2, 0, 1]
#                 sorted_indexes = np.argsort(NewPopfit)
#                 # 1-NewPop[2, :] ==> Z
#                 # 2-NewPop[0, :] ==> W
#                 # 3-NewPop[2, :] ==> Y
#                 newPopPos = NewPop[sorted_indexes[-1], :]

#             newPopPos = space_bound(newPopPos, upper_bound, lower_bound)
#             newPopFit = fitness_func(newPopPos)
#             print(f"Best is Complete for: {i+1} solution, {It+1} iteration".center(WIDTH))
#             print('\n\n')

#             # Update the population fitness and position if the new one is better (Maximization)
#             if newPopFit > Population_Fit[i]:  # Use '>' for maximization
#                 Population_Fit[i] = newPopFit
#                 Population_Pos[i, :] = newPopPos

#         # Update the global best solution found (Maximization)
#         for i in range(N):
#             if Population_Fit[i] > BestF:  # Use '>' for maximization
#                 BestF = Population_Fit[i]
#                 BestX = Population_Pos[i, :].copy()

#         curve[It] = BestF

#     return BestF, BestX, curve

# # Define the bounds for the parameters
# ub = [4, 3, 2, 2, 1]   # Max values for: TrainableLayers, Optimizers (as index), BatchSize, Dropout, ModelName (as index)
# lb = [0, 0, 0, 0, 0]   # Min values for: TrainableLayers, Optimizers (as index), BatchSize, Dropout, ModelName (as index)

# # Usage example
# N = 10             # Population(Solutions) size # Note Solutions Must Be >= 4
# T = 1              # Number of iterations
# dim = len(ub)      # Dimensionality of the problem

# # Run APO
# BestF, BestX, curve = APO(N, T, lb, ub, dim, fitness_func)

# print("Best fitness value:", BestF)
# print("Best solution:", BestX)

In [ ]:
# curve

In [ ]:
# plt.figure(figsize=(10, 5))
# plt.plot(range(len(curve)), curve, label='Fitness over Iters',   color='blue')
# plt.title('Curve for Iterations')
# plt.xlabel('Iterations')
# plt.ylabel('Fitness')
# plt.legend()
# plt.show()

In [ ]:
# Trainable_Layers, Optimizer, Batch_Size, dropout_value, best_model_name = index_values(BestX)
# print(f"trainable_layers: {Trainable_Layers}\n".center(WIDTH))
# print(f"optimizer: {Optimizer().__class__.__name__}\n".center(WIDTH))
# print(f"batch_size: {Batch_Size}\n".center(WIDTH))
# print(f"dropout_value: {dropout_value}\n".center(WIDTH))
# print(f"best_model_name: {best_model_name}".center(WIDTH))

In [ ]:
# # Path to the working directory
# check_points = '/kaggle/working/Checkpoints'
# logs         = '/kaggle/working/Logs'

# # # Loop through all files and subdirectories in the working directory
# for weight, csv in zip(os.listdir(check_points), os.listdir(logs)):
#     file_path_weight = os.path.join(check_points, weight)
#     file_path_csv    = os.path.join(logs, csv)
#     try:
#         if os.path.isfile(file_path_weight) or os.path.islink(file_path_weight)\
#         or os.path.isfile(file_path_csv) or os.path.islink(file_path_csv):
#             os.unlink(file_path_weight)  # Remove the file or symbolic link
#             os.unlink(file_path_csv)  # Remove the file or symbolic link
#         elif os.path.isdir(file_path_weight) or os.path.isdir(file_path_csv):
#             shutil.rmtree(file_path_weight)  # Remove the directory and its contents
#             shutil.rmtree(file_path_csv)  # Remove the directory and its contents
#     except Exception as e:
#         print(f'Failed to delete {file_path_weight}. Reason: {e}')
#         print(f'Failed to delete {file_path_csv}. Reason: {e}')

# print("All files and subdirectories in '/kaggle/working/' have been removed.")

In [ ]:
model, history = run_model(5,
                           Adam,
                           8,
                           0,
                           None,
                           True,
                           10,
                           # save=True,
                           patience=2)

In [ ]:
from tensorflow.keras.models import model_from_json, save_model, load_model
from tensorflow.keras.callbacks import ModelCheckpoint
import shutil  # For creating zip files for download

# Save the model architecture to a JSON file
with open("model.json", "w") as json_file:
    json_file.write(model.to_json())


# To save the entire model (architecture and weights) to a single HDF5 file
model.save("entire_model.h5")

shutil.make_archive('entire_model', 'zip', '.', 'entire_model.h5')
# To load the entire model from the saved file
# loaded_model = load_model("entire_model.h5")

# To load the model from JSON and weights
# with open("model.json", "r") as json_file:
#     loaded_model = model_from_json(json_file.read())
# loaded_model.load_weights("model_weights.h5")

In [ ]:
from IPython.display import FileLink
FileLink('entire_model.zip') # your model name.

In [ ]:
# new_model = create_model(model_name,
#                          trainable_layers,
#                          dropout_value,
#                          save_weights=True)

# # Load the saved weights
# new_model.load_weights('best_weights.h5')  # Path to the saved weights

# # Compile the model
# model.compile(optimizer=Optimizer(), loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
from tensorflow.keras.utils import plot_model

# Generate and save a model architecture plot
# plot_model(model, to_file='model_architecture.png', show_shapes=True, show_layer_names=True)

In [ ]:
# Get the path to the first image in the directory
image_dir = '/kaggle/working/merged_images/0'
image_path = os.path.join(image_dir, os.listdir(image_dir)[0])

# Read the image
image = cv2.imread(image_path)

# Resize the image to the desired shape
image = cv2.resize(image, (224, 224))  # OpenCV uses (width, height) format for resizing

# Convert the image to a NumPy array and preprocess it for the model
image = np.array(image)
print(image.shape)

# Optionally, if your model expects the image in a specific format (e.g., scaled to [0, 1]),
# you may need to normalize it as follows:
image = image / 255.0  # Scale pixel values to [0, 1]

# Add a batch dimension (1, 224, 224, 3)
image = np.expand_dims(image, axis=0)

# Display the image (optional)
plt.imshow(image[0])  # Use image[0] to display the image without the batch dimension
plt.axis('off')  # Turn off axis labels
plt.show()

# Make predictions using the new model
result = model.predict(image)
print("Prediction Result:", result)

del image;    gc.collect()

In [ ]:
import os

# Specify the directory
working_dir = '/kaggle/working/'

# Iterate through files in the directory
for file_name in os.listdir(working_dir):
    # Check if the file is a .png file
    if file_name.endswith('.png'):
        # Construct full file path
        file_path = os.path.join(working_dir, file_name)
        # Remove the file
        os.remove(file_path)
        print(f"Deleted: {file_path}")

print("All .png images have been removed from the working directory.")

In [ ]:
# Training data over epochs
epochs = range(len(history.history['accuracy']))
train_accuracy = history.history['accuracy']
train_precision = history.history['precision']  # Ensure these are recorded in history
train_recall = history.history['recall']
train_loss = history.history['loss']

# Accuracy Plot
plt.figure(figsize=(8, 5))
plt.plot(epochs, train_accuracy, label='Train Accuracy', color='blue', marker='o')
plt.title('Training Accuracy Over Epochs')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.grid()

# Annotate training accuracy values
for i, acc in enumerate(train_accuracy):
    plt.text(i, acc + 0.01, f"{acc:.4f}", ha='center')

plt.tight_layout()
plt.savefig('training_accuracy.png')
plt.show()

# Precision Plot
plt.figure(figsize=(8, 5))
plt.plot(epochs, train_precision, label='Train Precision', color='purple', marker='o')
plt.title('Training Precision Over Epochs')
plt.xlabel('Epochs')
plt.ylabel('Precision')
plt.grid()

# Annotate training precision values
for i, prec in enumerate(train_precision):
    plt.text(i, prec + 0.01, f"{prec:.4f}", ha='center')

plt.tight_layout()
plt.savefig('training_precision.png')
plt.show()

# Recall Plot
plt.figure(figsize=(8, 5))
plt.plot(epochs, train_recall, label='Train Recall', color='orange', marker='o')
plt.title('Training Recall Over Epochs')
plt.xlabel('Epochs')
plt.ylabel('Recall')
plt.grid()

# Annotate training recall values
for i, rec in enumerate(train_recall):
    plt.text(i, rec + 0.01, f"{rec:.4f}", ha='center')

plt.tight_layout()
plt.savefig('training_recall.png')
plt.show()

# Loss Plot
plt.figure(figsize=(8, 5))
plt.plot(epochs, train_loss, label='Train Loss', color='red', marker='o')
plt.title('Training Loss Over Epochs')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.grid()

# Annotate training loss values
for i, loss in enumerate(train_loss):
    plt.text(i, loss + 0.01, f"{loss:.4f}", ha='center')

plt.tight_layout()
plt.savefig('training_loss.png')
plt.show()

In [ ]:
# Evaluate on test dataset
test_results = model.evaluate(test_dataset)
print(test_results)
test_loss = test_results[0]
test_accuracy = test_results[1]
test_precision = test_results[2]
test_recall = test_results[3]

# Test metrics for bar plot
test_metrics = ['Accuracy', 'Precision', 'Recall', 'Loss']
test_values = [test_accuracy, test_precision, test_recall, test_loss]

# Plot test metrics as bars
plt.figure(figsize=(8, 5))

# Bar plot with custom colors
bars = plt.bar(test_metrics, test_values, color=['#003049', '#219ebc', '#f77f00', '#d62828'])

# Set y-axis limits and labels
plt.ylim(0, 1)  # Adjust for metric ranges (may vary)
plt.title('Test Metrics', fontsize=14)
plt.ylabel('Value', fontsize=12)

# Annotate each bar with the corresponding value
for bar in bars:
    yval = bar.get_height()
    plt.text(bar.get_x() + bar.get_width() / 2, yval + 0.02, f'{yval:.4f}', ha='center', va='bottom', fontsize=10)

# Tight layout for better spacing
plt.tight_layout()

# Save and display the plot
plt.savefig('test_metrics_bar_custom_colors.png')
plt.show()

In [ ]:
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

# Get true labels and predicted labels for the test data
y_true = np.concatenate([y for x, y in test_dataset], axis=0).argmax(axis=1)
y_pred = np.concatenate([model.predict(x) for x, y in test_dataset], axis=0).argmax(axis=1)

# Generate the confusion matrix
conf_matrix = confusion_matrix(y_true, y_pred)
disp = ConfusionMatrixDisplay(confusion_matrix=conf_matrix)

# Plot the confusion matrix
plt.figure(figsize=(8, 6))
disp.plot(cmap='Blues')
plt.title('Confusion Matrix')
plt.grid(False)
plt.tight_layout()
plt.savefig('confusion_matrix.png')
plt.show()

In [ ]:
# del model,\
#     history,\
#     results,\
#     test_loss,\
#     test_accuracy,\
#     epochs,\
#     train_loss,\
#     train_accuracy,\
# #     val_loss,\
# #     val_accuracy
# gc.collect()